###  Use watsonx, and `ibm/granite-13b-instruct-v2` 
To analyze customer satisfaction based on the feedback provided by customers who are using electronic devices manufactured by a specific company.

### Install and import the `datasets` and dependecies

In [63]:
!pip install datasets | tail -n 1
!pip install "scikit-learn==1.3.2" | tail -n 1
!pip install -U ibm-watsonx-ai | tail -n 1

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.


In [64]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Please enter your WML api key (hit enter): "),
)

Please enter your WML api key (hit enter): ········


### Defining the project id
The Foundation Model requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

In [65]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

In [66]:
import pandas as pd

# URL of the raw CSV file from GitHub (replace with your actual URL)
url = "https://raw.githubusercontent.com/rahul-bhave/watsonx_ai_maverics/main/electronic_device_customer_feedback.csv"

# Load the CSV file directly from the GitHub URL
df = pd.read_csv(url)

# Extract only the required columns
data = df[['Customer_Service', 'Satisfaction']]

# Display the DataFrame
print(data)


                                      Customer_Service  Satisfaction
0                                  The device is good.             1
1    Absolutely love this VR headset! It’s a game-c...             1
2        The device works perfectly, no issues so far!             1
3    The screen resolution on this laptop is amazin...             0
4    Picture quality is stunning! Gaming feels supe...             1
..                                                 ...           ...
495  Fantastic device, really happy with this purch...             1
496  Great product for the price! I use it daily an...             1
497  Fantastic sound quality, perfect for long list...             1
498  Fantastic sound quality, perfect for long list...             1
499  Disappointed with the camera performance in lo...             0

[500 rows x 2 columns]


Examine downloaded data.

In [67]:
data.head()

,Customer_Service,Satisfaction
0,The device is good.,1
1,Absolutely love this VR headset! It’s a game-c...,1
2,"The device works perfectly, no issues so far!",1
3,The screen resolution on this laptop is amazin...,0
4,Picture quality is stunning! Gaming feels supe...,1


Prepare train and test sets.

In [68]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.5)
comments = list(test.Customer_Service)
satisfaction = list(test.Satisfaction)

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

We will list out all avilable models


In [69]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL', 'MERLINITE_7B', 'GRANITE_20B_CODE_INSTRUCT', 'GRANITE_34B_CODE_INSTRUCT', 'GRANITE_3B_CODE_INSTRUCT', 'GRANITE_7B_LAB', 'GRANITE_8B_CODE_INSTRUCT', 'LLAMA_3_70B_INSTRUCT', 'LLAMA_3_8B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01']


#### Get the model id
Get the `model_id` that will be used in next steps

In [70]:
model_id = ModelTypes.GRANITE_13B_INSTRUCT_V2

### Defining the model parameters

Define model parameters such as min, max new tokens, decoding method selected Greedy as well as repetition penalty is mentioned.

In [71]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.MIN_NEW_TOKENS: 0,
    GenParams.MAX_NEW_TOKENS: 1,
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.REPETITION_PENALTY: 1
}

### Initialize the model


In [72]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

### Model's details

In [73]:
model.get_details()

{'model_id': 'ibm/granite-13b-instruct-v2',
 'label': 'granite-13b-instruct-v2',
 'provider': 'IBM',
 'source': 'IBM',
 'functions': [{'id': 'prompt_tune_inferable'},
  {'id': 'prompt_tune_trainable'},
  {'id': 'text_generation'}],
 'short_description': 'The Granite model series is a family of IBM-trained, dense decoder-only models, which are particularly well-suited for generative tasks.',
 'long_description': 'Granite models are designed to be used for a wide range of generative and non-generative tasks with appropriate prompt engineering. They employ a GPT-style decoder-only architecture, with additional innovations from IBM Research and the open community.',
 'input_tier': 'class_1',
 'output_tier': 'class_1',
 'number_params': '13b',
 'min_shot_size': 0,
 'task_ids': ['question_answering',
  'summarization',
  'classification',
  'generation',
  'extraction'],
 'tasks': [{'id': 'question_answering', 'ratings': {'quality': 3}},
  {'id': 'summarization',
   'ratings': {'quality': 2}

<a id="predict"></a>
## Analyze the satisfaction

#### Prepare prompt and generate text

In [74]:
instruction = """Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:Returned the product as it didn’t match the description on the website.
Satisfied:no"""

In [75]:
prompt1 = "\n".join([instruction, "Comment:" + comments[2], "Satisfied:"])
print(prompt1)

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:Returned the product as it didn’t match the description on the website.
Satisfied:no
Comment:Returned the product as it didn’t match the description on the website.
Satisfied:


Analyze the sentiment for a sample of zero-shot input from the test set.

In [76]:
print(model.generate_text(prompt=prompt1))

no


### Calculate the accuracy

In [77]:
sample_size = 10
prompts_batch = ["\n".join([instruction, "Comment:" + comment, "Satisfied:"]) for comment in comments[:10]]
results = model.generate_text(prompt=prompts_batch)

In [78]:
print(prompts_batch[0])

Determine if the customer was satisfied with the experience based on the comment. Return simple yes or no.
Comment:Returned the product as it didn’t match the description on the website.
Satisfied:no
Comment:Sound is not proper when you play jazz.
Satisfied:


<a id="score"></a>
## Score the model

In [79]:
from sklearn.metrics import accuracy_score

label_map = {0: "no", 1: "yes"}
y_true = [label_map[sat] for sat in satisfaction][:sample_size]

print('accuracy_score', accuracy_score(y_true, results))

accuracy_score 0.9


In [80]:
print('true', y_true, '\npred', results)

true ['no', 'no', 'no', 'yes', 'yes', 'no', 'yes', 'yes', 'yes', 'yes'] 
pred ['no', 'no', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes', 'yes']


### References:
1. https://dataplatform.cloud.ibm.com/exchange/public/entry/view/61c1e967-8d10-44bb-a846-cc1f27e9e69a?context=wx
2. https://github.com/IBM/watson-machine-learning-samples/blob/master/README.md
